# Solving Systems of Equations - Gauss Elimination

In this notebook we will begin learning how to solve systems of equations. That is
$$\begin{align*} 
7x_1 + 2x_2 - 3x_3 &= 12 \\
3x_1 - 8x_2 + 2x_3 &= 4 \\
-6x_1 + 2x_2 +9x_3 &= 9
\end{align*}$$

which we will write in matrix-vector form from now on as $\mathbf{A}\mathbf{x}=\mathbf{b}$

$$
\underbrace{\begin{bmatrix}
7 & 2 & -3 \\
3 & -8 & 2 \\
-6 & 2 & 9 \\
\end{bmatrix}}_{\mathbf{A}}
\underbrace{\begin{bmatrix}
x_1     \\
x_2     \\
x_3     \\
\end{bmatrix}}_{\mathbf{x}}
=
\underbrace{\begin{bmatrix}
12     \\
4     \\
9     \\
\end{bmatrix}}_{\mathbf{b}}
$$

The idea behind Gauss Elimination is to transform the system above ($\mathbf{A}\mathbf{x}=\mathbf{b}$) into an \emph{upper triangular} system ($\mathbf{A^*}\mathbf{x}=\mathbf{b^*}$) as seen below

$$
\underbrace{\begin{bmatrix}
A_{11} & A_{12} & A_{13} \\
0 & A^*_{22} & A^*_{23} \\
0 & 0 & A^*_{33} \\
\end{bmatrix}}_{\mathbf{A^*}}
\underbrace{\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
\end{bmatrix}}_{\mathbf{x}}
=
\underbrace{\begin{bmatrix}
b^*_1 \\
b^*_2 \\
b^*_3 \\
\end{bmatrix}}_{\mathbf{b^*}}
$$

where any variable with an asterisk such as $\mathbf{A^*}$ indicates they have been modified from the original system.

Once you triangulate the system, you can easily solve for the last variable as
$$x_3 = A^*_{33} / b^*_3$$
Now that you have solved for $x_3$, you can move your attention to the next row up to solve for $x_2$. Notice how we are moving backwards. Expanding the second equation we get
$$A^*_{22}x_2 + A^*_{23}x_3 = b^*_2$$
and then re-arranging to solve for $x_2$ we get
$$x_2 = \frac{b^*_2 - A^*_{23}x_3}{A^*_{22}}$$
and we can repeat this same idea for each subsequent equation/row above until we find all of the $x_i$. This formula is generalized as
$$ x_i = \frac{b^*_i - \sum\limits_{j=i+1}^{N} A^*_{ij}x_j}{A^*_{ii}} $$
